In [1]:
# Import the libraries
from pystac.extensions.eo import EOExtension as eo
import pystac_client
import planetary_computer
from odc.stac import configure_rio, stac_load
import dask.distributed
import dask.utils
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from IPython.display import display


In [2]:
# Set up Dask client for parallel processing
client = dask.distributed.Client()
configure_rio(cloud_defaults=True, client=client)

# Configure rio with dynamic resolution
resolution = 20
memory_limit = dask.utils.parse_bytes(client.cluster.workers[0].memory_manager.memory_limit)
SHRINK = 4
if memory_limit < dask.utils.parse_bytes("4G"):
    SHRINK = 8  # Adjust chunk size if memory is limited

resolution = resolution * SHRINK


In [3]:
# Define the area of interest (AOI) for Lake Michigan
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [-88.2, 43.0],  # Lower-left corner
            [-86.1, 43.0],  # Lower-right corner
            [-86.1, 45.0],  # Upper-right corner
            [-88.2, 45.0],  # Upper-left corner
            [-88.2, 43.0],  # Closing the polygon
        ]
    ],
}

#  time span of 10 years
time_of_interest = "2013-06-01/2023-06-01"

# Query the catalog for the data
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=area_of_interest,
    datetime=time_of_interest
)
items = list(search.items())
print(f"Returned {len(items)} Items")


Returned 7684 Items


In [4]:
# Load the data from the catalog with dynamic chunking and handle missing data
xx = stac_load(
    items,
    chunks={"x": 1024 * SHRINK, "y": 1024 * SHRINK},  # Dynamically adjust chunk size
    patch_url=planetary_computer.sign,
    resolution=resolution,
    dtype="uint16",  # Handle missing data by marking nodata values
    nodata=0
)

# Display loaded data
print(f"Bands: {','.join(list(xx.data_vars))}")
display(xx)


Bands: AOT,B01,B02,B03,B04,B05,B06,B07,B08,B09,B11,B12,B8A,SCL,WVP,visual


<xarray.Dataset> Size: 155GB
Dimensions:      (y: 1938, x: 1937, time: 1291)
Coordinates:
  * y            (y) float64 16kB 5e+06 5e+06 5e+06 ... 4.69e+06 4.69e+06
  * x            (x) float64 15kB 3.001e+05 3.002e+05 ... 6.097e+05 6.098e+05
    spatial_ref  int32 4B 32616
  * time         (time) datetime64[ns] 10kB 2015-08-10T16:39:46.027000 ... 20...
Data variables: (12/16)
    AOT          (time, y, x) uint16 10GB dask.array<chunksize=(1, 1938, 1937), meta=np.ndarray>
    B01          (time, y, x) uint16 10GB dask.array<chunksize=(1, 1938, 1937), meta=np.ndarray>
    B02          (time, y, x) uint16 10GB dask.array<chunksize=(1, 1938, 1937), meta=np.ndarray>
    B03          (time, y, x) uint16 10GB dask.array<chunksize=(1, 1938, 1937), meta=np.ndarray>
    B04          (time, y, x) uint16 10GB dask.array<chunksize=(1, 1938, 1937), meta=np.ndarray>
    B05          (time, y, x) uint16 10GB dask.array<chunksize=(1, 1938, 1937), meta=np.ndarray>
    ...           ...
    B11          (time, y, x) uint16 10GB dask.array<chunksize=(1, 1938, 1937), meta=np.ndarray>
    B12          (time, y, x) uint16 10GB dask.array<chunksize=(1, 1938, 1937), meta=np.ndarray>
    B8A          (time, y, x) uint16 10GB dask.array<chunksize=(1, 1938, 1937), meta=np.ndarray>
    SCL          (time, y, x) uint16 10GB dask.array<chunksize=(1, 1938, 1937), meta=np.ndarray>
    WVP          (time, y, x) uint16 10GB dask.array<chunksize=(1, 1938, 1937), meta=np.ndarray>
    visual       (time, y, x) uint16 10GB dask.array<chunksize=(1, 1938, 1937), meta=np.ndarray>

In [5]:
# Function to convert data to float and handle missing nodata values
def to_float(xx, nodata_value=None):
    _xx = xx.astype("float32")  # Convert data to float32 for precision
    if nodata_value is None:
        nodata_value = _xx.attrs.pop("nodata", None)  # Fetch nodata value if exists
    if nodata_value is not None:
        return _xx.where(xx != nodata_value)  # Replace nodata with NaN
    return _xx

# Convert specific bands to float32 and handle missing data
b05 = to_float(xx.B05)  # Red-Edge band
b04 = to_float(xx.B04)  # Red band


In [6]:
# Calculate NDCI with small constant to avoid division by zero
ndci = (b05 - b04) / (b05 + b04 + 1e-6)

# Apply Min-Max normalization to scale NDCI between 0 and 1
ndci = (ndci - ndci.min()) / (ndci.max() - ndci.min())

# Display the calculated NDCI
display(ndci)


<xarray.DataArray (time: 1291, y: 1938, x: 1937)> Size: 19GB
dask.array<truediv, shape=(1291, 1938, 1937), dtype=float32, chunksize=(1, 1938, 1937), chunktype=numpy.ndarray>
Coordinates:
  * y            (y) float64 16kB 5e+06 5e+06 5e+06 ... 4.69e+06 4.69e+06
  * x            (x) float64 15kB 3.001e+05 3.002e+05 ... 6.097e+05 6.098e+05
    spatial_ref  int32 4B 32616
  * time         (time) datetime64[ns] 10kB 2015-08-10T16:39:46.027000 ... 20...

In [10]:
# Efficient concatenation of NDCI across the time dimension
ndci_comp = xr.concat([ndci.isel(time=i) for i in range(len(ndci))], dim="time").compute()

# Display concatenated time series
print(ndci_comp)


2024-11-10 17:43:24,107 - distributed.worker - ERROR - Compute Failed
Key:       ('B05-7977647e00c3fed9b0845031478e3886', 985, 0, 0)
State:     executing
Function:  _dask_loader_tyx
args:      ([[<odc.loader._rio.RioReader object at 0x117027a30>, <odc.loader._rio.RioReader object at 0x1213ef100>, <odc.loader._rio.RioReader object at 0x1213effa0>, <odc.loader._rio.RioReader object at 0x116a35390>, <odc.loader._rio.RioReader object at 0x116fb2b60>, <odc.loader._rio.RioReader object at 0x116fb1930>, <odc.loader._rio.RioReader object at 0x117025600>, <odc.loader._rio.RioReader object at 0x1213ef8e0>, <odc.loader._rio.RioReader object at 0x1170255d0>, <odc.loader._rio.RioReader object at 0x116fb1540>, <odc.loader._rio.RioReader object at 0x1213ec730>, <odc.loader._rio.RioReader object at 0x1213ecd60>, <odc.loader._rio.RioReader object at 0x1213ee860>, <odc.loader._rio.RioReader object at 0x1213efd00>, <odc.loader._rio.RioReader object at 0x116a34fd0>, <odc.loader._rio.RioReader object at 0x

CRSError: The EPSG code is unknown. PROJ: internal_proj_create_from_database: /opt/anaconda3/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 3 is expected. It comes from another PROJ installation.

In [ ]:
# Here I am trying to visualize the NDCI index for verification. This step is additional and not part of the notebook provided to us 
_ = ndci.isel(time=335).compute().plot.imshow(size=7, aspect=1.2, interpolation="bicubic")

# Loop through time steps to visualize the time series
for i in range(0, len(ndci), 50):  # Every 50th time step
    plt.figure()
    ndci.isel(time=i).compute().plot.imshow(size=7, aspect=1.2, interpolation="bicubic")
    plt.title(f"NDCI - Time step {i}")
    plt.show()


In [ ]:
# Save the NDCI data to a NetCDF file
output_path = "/mnt/data/ndci_data.nc"
ndci.to_netcdf(output_path)  # Replace `ndci` with the variable holding the NDCI data
print(f"NDCI data saved to {output_path}")
